## Домашнее задание «Рекомендации на основе скрытых факторов»

Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)

In [75]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

%matplotlib inline

from implicit.als import AlternatingLeastSquares

In [116]:
bookmarks = pd.read_csv('bookmarks.dat',sep='\t', encoding = "ISO-8859-1",usecols=['id','title','url'])
user_bookmark = pd.read_csv('user_taggedbookmarks.dat',sep='\t', encoding = "ISO-8859-1",usecols=['userID' ,'bookmarkID'])

In [117]:
bookmarks.head()

,id,title,url
0,1,IFLA - The official website of the Internation...,http://www.ifla.org/
1,2,gcdp-e.pdf (application/pdf Object),http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf
2,7,EdSelect,http://www.edselect.com/
3,8,Cool Canada (Collections Canada),http://www.collectionscanada.gc.ca/cool/index-...
4,9,Kidsreads.com,http://www.kidsreads.com/


In [78]:
user_bookmark.head()

,userID,bookmarkID
0,8,1
1,8,2
2,8,7
3,8,7
4,8,7


In [79]:
print("user with bookmark count = {},bookmark = {}".format(user_bookmark.shape[0],bookmarks.shape[0]))

user with bookmark count = 437593,bookmark = 69226


In [144]:
user_bookmark.drop_duplicates(inplace=True)

In [145]:
user_bookmark['user_id'] = user_bookmark['userID'].astype("category").cat.codes
user_bookmark['bookmark_id'] = user_bookmark['bookmarkID'].astype("category").cat.codes

In [139]:
user_bookmark.head()

,userID,bookmarkID,user_id,bookmark_id
0,8,1,0,0
1,8,2,0,1
2,8,7,0,2
3,8,7,0,2
4,8,7,0,2


In [84]:
data = user_bookmark [['user_id','bookmark_id']]

In [86]:
index_lookup = user_bookmark[["bookmarkID","bookmark_id"]].drop_duplicates();

In [122]:
bookmark_dict = {}

for index, row in tqdm_notebook(index_lookup.iterrows()):
    bookmark_dict[row.bookmark_id]=bookmarks[bookmarks.id==row.bookmarkID].url

In [90]:
users_list = list(np.sort(data.user_id.unique()))
bookmark_list = list(np.sort(data.bookmark_id.unique()))
flag = [1 for i in range(data.shape[0])]

In [92]:
rows = data.user_id.astype(int)
cols = data.bookmark_id.astype(int)

In [93]:
data_sparse = sparse.csr_matrix((flag, (cols, rows)), shape=(len(bookmark_list), len(users_list)))

In [94]:
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|██████████| 15.0/15 [00:07<00:00,  2.28it/s]


In [153]:
userid = 3

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [154]:
recommendations

[(192, 0.40453607),
 (232, 0.36716503),
 (219, 0.35856515),
 (221, 0.3578941),
 (223, 0.3481099),
 (206, 0.34804004),
 (217, 0.34659103),
 (193, 0.3450229),
 (205, 0.34449822),
 (229, 0.3407296)]

## Source:

In [155]:
source = user_bookmark[user_bookmark.user_id==3].bookmark_id.tolist()

In [156]:
for r in source:
    print(bookmark_dict[r])

116    http://bridgeurl.com/
Name: url, dtype: object
117    http://mashable.com/2010/11/07/new-social-medi...
Name: url, dtype: object
118    http://www.connectsafely.org/Safety-Advice-Art...
Name: url, dtype: object
119    http://www.mathopenref.com/tocs/constructionst...
Name: url, dtype: object
120    http://clifmims.wetpaint.com/page/Modules
Name: url, dtype: object
121    http://technology4kids.pbworks.com/w/page/2429...
Name: url, dtype: object
122    http://data.un.org/
Name: url, dtype: object
123    http://voices.washingtonpost.com/answer-sheet/...
Name: url, dtype: object
124    http://1toolatatime.wikispaces.com/QR Codes
Name: url, dtype: object
125    http://www.webexhibits.org/daylightsaving/inde...
Name: url, dtype: object
126    http://www.nytimes.com/2010/11/08/technology/0...
Name: url, dtype: object
127    http://lisadawley.wordpress.com/2010/11/06/pro...
Name: url, dtype: object
128    http://news.wfu.edu/2010/11/07/ipad-more-than-...
Name: url, dtype: object
129   

## Our reccomendations:

In [157]:
for r in recommendations:
    print(bookmark_dict[r[0]])

192    http://cssgrid.net/
Name: url, dtype: object
232    http://infosthetics.com/
Name: url, dtype: object
219    http://www.shopify.com/
Name: url, dtype: object
221    http://www.webalys.com/design-interface-applic...
Name: url, dtype: object
223    http://www.uxbooth.com/
Name: url, dtype: object
206    http://slides.html5rocks.com/#slide1
Name: url, dtype: object
217    http://www.noupe.com/design/an-explosion-of-ad...
Name: url, dtype: object
193    http://designmoo.com/
Name: url, dtype: object
205    http://www.uxmag.com/
Name: url, dtype: object
229    http://moritz.stefaner.eu/
Name: url, dtype: object
